# Exercise 4
In this exercise, we will control a 2D drone depicted in the figure below <br>
<img src='quadrotor.png' width="300">

Consider the following simplified linear model of the drone
$$\begin{align}
    x_{n+1} = A
    x_n + B u_n
\end{align}$$

where the components of the state $x_n$ correspond to the horizontal position and velocity of the quadrotor, its vertical position and velocity and its orientation and angular velocity. The control vector
$u_n$ contains the forces produced by the rotors (our control inputs).
The module ```quadrotor.py``` defines the matrices $A$ and $B$ and code to generate a simulation of the drone.

In [43]:
## what we need to do computation and display the drone
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt

import quadrotor

In [44]:
# the matrices A and B are already defined in the quadrotor module
print(f'A =\n {quadrotor.A}')
print(f'B =\n {quadrotor.B}')


A =
 [[ 1.      0.01    0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.     -0.0981  0.    ]
 [ 0.      0.      1.      0.01    0.      0.    ]
 [ 0.      0.      0.      1.      0.      0.    ]
 [ 0.      0.      0.      0.      1.      0.01  ]
 [ 0.      0.      0.      0.      0.      1.    ]]
B =
 [[ 0.     0.   ]
 [ 0.     0.   ]
 [ 0.     0.   ]
 [ 0.02   0.02 ]
 [ 0.     0.   ]
 [ 0.015 -0.015]]


We want to generate a control input will move the drone towards the point $(3,3)$ starting from $(0,0)$. In order to
do so, we define a cost that penalizes the distance to the goal at each time step while minimizing velocities and the amount of trust needed,
i.e. we want to solve the following optimal control problem
$$\begin{align}
&\min_{x_0, u_0, x_1, u_1, \cdots} \frac{1}{2}\sum_{n=0}^{N} (x_n - x_{desired})^T Q (x_n - x_{desired}) + u_n^T R u_n\\
\textrm{subject to}\ \ & x_{n+1} = A x_{n} + B u_n\\
& x_0 = [0,0,0,0,0,0]^T
\end{align}$$
where $x_{desired} = [3,0,3,0,0,0]^T$

1. Write down the KKT conditions for the problem
2. Write code to solve the problem for N=500 time steps (you will need to find diagonal matrices $Q>0$ and $R>0$ to create a nice movement) by solving the KKT system of linear equations. Do NOT inverse the KKT matrix, instead use the [NumPy solve function](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html) which is numerically more stable and efficient.
4. Show plots of all the states of the robot as a function of time
5. Show plots of the optimal control as a function of time

In [45]:
# we define a few constants
xdim = 6 # dimension of the state (p,v)
udim = 2 # dimension of the control (u)
N = 500 # number of steps
nvars = N*udim + (N+1)*xdim # total number of variables
dt = 0.01 #delta t

## We define the cost
Q = np.eye(xdim)
Q[0,0] = Q[2,2] = Q[4,4] = 100.  # position penalty
Q[1,1] = Q[3,3] = Q[5,5] = 0.90   # velocity penalty
R = np.eye(udim) * 0.001

# Desired State
x_desired = np.array([3, 0, 3, 0, 0, 0])


## we create G
def get_cost_matrix_G(Q, R):
    '''
    we are making this a function so we can reuse this later in the next cells when we play with costs
    '''
    G = np.zeros((nvars, nvars))
    for i in range(N):
        G[i*(xdim+udim):i*(xdim+udim)+xdim,i*(xdim+udim):i*(xdim+udim)+xdim] = Q
        G[i*(xdim+udim)+xdim:i*(xdim+udim)+xdim+udim, i*(xdim+udim)+xdim:i*(xdim+udim)+xdim+udim] = R
    G[N*(xdim+udim):N*(xdim+udim)+xdim,N*(xdim+udim):N*(xdim+udim)+xdim] = Q
    return G

G = get_cost_matrix_G(Q,R)

##### CREATE THE KKT SYSTEM #####
## create the linear equality constraint
C = np.zeros([xdim * (N+1), nvars])
d = np.zeros([xdim * (N+1)])

C[0:6, 0:6] = np.eye(xdim)
d[0] = 0. # initial position of the car

A = quadrotor.A
B = quadrotor.B

for i in range(N):
    ##u
    C[(i+1)*xdim:(i+2)*xdim, i*(xdim+udim)+xdim:i*(xdim+udim)+xdim+2] = B
    ##x n+1
    C[(i+1)*xdim:(i+2)*xdim, (i+1)*(xdim+udim):(i+1)*(xdim+udim)+xdim] = -np.eye(xdim)
    ## xn
    C[(i+1)*xdim:(i+2)*xdim, i*(xdim+udim):(i)*(xdim+udim)+xdim] = A

## create the KKT matrix
nconstraints = C.shape[0]
KKT = np.zeros((nvars+nconstraints, nvars+nconstraints))

KKT[0:nvars, 0:nvars] = G
KKT[nvars:, 0:nvars] = C
KKT[0:nvars, nvars:] = C.T

f = np.zeros((nvars+nconstraints,1))
# Set the desired final state
f[N*(xdim+udim):N*(xdim+udim)+xdim, 0] = Q @ x_desired

# Set the initial state constraint
x0 = np.zeros(xdim)
f[nvars:nvars+xdim, 0] = x0

# Add small regularization term for numerical stability
regularization = 1e-6
KKT += regularization * np.eye(KKT.shape[0])

# Solve the KKT system
x = np.linalg.solve(KKT, f)

# Extract states and controls
states = x[:N*xdim].reshape(N, xdim)
controls = x[N*xdim:N*(xdim+udim)].reshape(N, udim)

# Plot results
t = np.arange(N) * dt




In [46]:
# we can display the behavior of the drone based on a given control input

# we can also start at x0 = 0 and generate a random control of 0 for 300 time steps
x0 = np.zeros((6,1))
u = u = controls.T

quadrotor.animate_robot(x0, u, goal=[3,3])


# # Print final state
# print("Final state:", states[-1])

# # Prepare inputs for quadrotor animation
# x0 = np.zeros((6,1))  # Initial state
# u = controls.T  # Transpose to match expected shape (2,N)

# # Animate the quadrotor
# quadrotor.animate_robot(x0, u, goal=[3,3])

<source type="video/mp4" src="data:video/mp4;base64,AAAAIGZ0eXBNNFYgAAACAE00ViBpc29taXNvMmF2YzEAAAAIZnJlZQAAJ6ZtZGF0AAACrwYF//+r
3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyByMzA2MCA1ZGI2YWE2IC0gSC4yNjQvTVBF
Ry00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAyMSAtIGh0dHA6Ly93d3cudmlkZW9sYW4u
b3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFs
eXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVk
X3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBk
ZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE5
IGxvb2thaGVhZF90aHJlYWRzPTMgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50
ZXJsYWNlZD0wIGJsdXJheV9jb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBi
X3B5cmFtaWQ9MiBiX2FkYXB0PTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29w
PTAgd2VpZ2h0cD0yIGtleWludD0yNTAga2V5aW50X21pbj0xMCBzY2VuZWN1dD00MCBpbnRyYV9y
ZWZyZXNoPTAgcmNfbG9va2FoZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0w
LjYwIHFwbWluPTAgcXBtYXg9NjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAA
EXdliIQAEf/+94gfMstp+TrXchHnrS6tH1DuRnFepL3+IAAAAwAAAwACHHgr8h1jA36boAAANaAG
nDrYXyDttuTzAJybKbjYDdPrCFjUKWupGhi4jXXEJA89Q6Zn33H4Fv/q9RH8BCnmRMbhoCA/JXZ9
HyMLdr9bnUhW+7vLTMSLC18YaGWT/qmDBhUQzv5HiDtjFO9FOl/qju20IhrpDMgMbFwMyTXqHsaP
8VTwxCrdyHWvaEdLkEloVdMX7+j69RLccpMwYEgAfeXG61171o8ifS427jwgzVo1tPzlj50LeiKV
EgzNjk/mgdZMJaC8200Gcg9WuoIZp1ruckQ36Gn7+Snn2WwB8/RwkBUPzahcRpGvAX0W7YjSfH6s
oQiVF5wlthfs3Rv5v9SCnGFOgwvj+TWoTrj2RqRzMQUMGhWTBDE6SZ8uf5xnwlJtm9VSFA6yvlxz
gFetBoXD9Df16ReFiqTeYw5Xh8vOzv5IF3eJ153yH4hMm39UP24A7QArSk46x+858Bz8zwpQ+fR0
uIAIHFJGdhzFCi/Gt0hAL65/wNj/5bo2NkEWg0gDDdvnGwgomP1jUgXVv4mcfJYV9eOpsDFKsX1w
GJWA8WCyCh/B/zfkBsVmzo7fDky9j80176crLLagpUsc7mdlzOOqVIG/hiiCKvS3++RM3vldrP7f
tcsngNp/3lO6iX5yKc0BGM3t3ohSoTh81HBwC38Xc5OcVccGr4JkKlR2M/P45yut9miQqUIf0Quv
qpsyMHJFklb76vfcSOL8hwHfFMkise+eaAEZQR64UHrCoU4l3bW9xNHE3kar8Dcw2hT8sXL2D5sq
eRT+KPgGhyh4Lqjo6TQUZPUAsXD0VSz7D413DMDvWu2lvRksszoNrgNgO7G4JM3Nhc05b94Ggxf6
pvw21RcPa9Ovg49YKF0XTL8x9EVRsKoEfy9LZnP8Sxjr7rR7EGFbyzDZ2AcBc/6SFvZ2jwdhiwau
yEDj/qR0z/Pa+mDY8du3dBDARlvFr6zRtI7ypDvd23csqf0nSLLmAke9H3yBq2vEsopgNcjA4Otc
SaqwjFrnQxlmZVgF9y3Zi2Md07Yof0lDPVE2VTIJ+/eY//Ypfa5OJhKT91+KXiXMgpM5mzocmKAE
gq2HpHFvlIh8A7yDaYEmNlFJAIIsj21LOGIddblaqoYdRpYWICRoXbLUhwM9CAFrgqryF0n+2znj
38vmEEt30MLqC+LjJEoDU9hNM0yqBhraezs+tNOKskBKZC4/DNNnB+g/MJg+mpm8z76wyrePWZjr
ZKCMl91YjfLo47vOnw3RJYG9Y7cjT8tVBTXDTfBjHbCsU15isD3wBjPX1WMUvjYQKxqrdiBRN+e5
TLX4RF9Uq5vb4Qh1x40BnTFQ5mfpxYsiz3+1YpsTeDH5fLoa+eH+u2FnlTJoL/+2WKLzAJ34Skk2
/0hXp2HlOrxpCid40YZz21C4zAkjynNpJfC+pkc3W+UnsvNMKfqaS34JGI/VWSkRmjrAct9CSZlr
IZLZ8IHdB+zy+pFHNy+Mge/eD6r9ExoX4m1yRvUYx4pmsZrAVphQKXg50Vmo6LfnCzTitXIhcOSz
aD7NCaElDCw39guwfXFPK5LfWbLT2GjNs6UgCUBA/8nilz1CmpZqpVYkY0Ad2R+rb1EEy36wVIdP
x1AdLpcGRq+gprBaJkpto8+Z7IUOwQt5jE1d/0yHf+M16WXlisWzccAqtRbY99jW9DPgksvZyyoB
Z7oivXXv6+KA0ax+HuuAo6rdDWiQWLpGGo84IJqhsGY+Yn8w/emruTQov+nd+0jmVqJdiLKp4nr8
y7gQhh9mxFpen9qSPJkCmb+lexCkKsoacmn2NnfsAHrPHz+4xZhZMk/KdQ9f7VjqhKhp2Xqbs9C5
cAu8wga/Kw6xXdz7drS22kjVvmDt+cFQPwT1ZK5C+Dnm9RZUT6EZWgrCIYt5c//sRmotccnSThQ+
xPkpylyOUDNM22nq3akZRdaHZvkqkpUvj/nfz+ylsnZfkPM8L02Kl2KgABd33k532Z/69lsE9BL5
IX/T+9GJsj1Ff6ArS8eCe0kTcVHz4SUfnLvPMrGwh/OchE7wUxI+u0AjBE+ia46vU7VCujfyDYF7
hucYzJrQsJuyvmRBTriyr6U7G4weknlLoUxBt37coJGD0yglQjzAnwzfIaDnPsG+Ckot/QVvTps7
j6cemg0HUKcrbfEfkGf0zuix9DjfB0kW9MGDb4iwMVkPvgjn8Ob9PzPVXu4Tr1QZJAtRpoZduKPY
XxGnVJ7AnfkdxLf/8Et1gUodlPATgiCJWdhPP63Y7Aa63JzYv7QVY308bFxGgapjOCiP9IkbImTw
ncTx6BTMRVdW3/4Aw7Tz0XtoQBJ4X/wqPKKFZjlh9CMxkyHlhtD7IAxKHyp5lpKXudGWGP8yNOWE
II8Bik/l1Wv4d9ynw7Rbssy+n/1WeJ9dz/VAO+ufZKBjBGONiB45OfwY8nHSxsWPQoXNDaN3U2GM
qZSu53yTL6kGBUnZgl11N/pwyxcUwEgtSnHv+oRezXhzDSsDlf286WlIZtRN61XGyDeN7vNNscOv
7gboOup8lTuOFstHavNwlFiXVWqHQ2nlWcbPUfF+fyeMRiMvo7mP/DFaH5jVIyhLe4cZxUsd/nhW
LHQmWy7GUSxP9iZbVQ74bMAT50EDD59yiQ+XGmgLeG4yZBM/eJ/H2RqvxX2NY6EREvCEUSEhRLEf
7qgm9eCy5XZxp+eaYyRNn7yYqfEQgSvM/y0s4yjKtXIfuWivADT3oJNAT/JOxFuLgHNUuziYi4F5
8ijhVXXr9irpKv7acIkeyMp2XvZoc//KzYKNHuMxZ1m6iuv+pdBgvuo77gn21ymhiHmLL/+78eKv
jy1W4GL3AAq5FYHiOZw/P